**Проект по А/B-тестированию**

**Задача** — провести оценку результатов A/B-теста. 

- Оцените корректность проведения теста и проанализируйте его результаты.

Чтобы оценить корректность проведения теста:
- удостоверьтесь, что нет пересечений с конкурирующим тестом и нет пользователей, участвующих в двух группах теста одновременно;
- проверьте равномерность распределения пользователей по тестовым группам и правильность их формирования.


**Техническое задание**
Название теста: recommender_system_test ;
- Группы: А (контрольная), B (новая платёжная воронка);
- Дата запуска: 2020-12-07;
- Дата остановки набора новых пользователей: 2020-12-21;
- Дата остановки: 2021-01-04;
- Ожидаемое количество участников теста: 15% новых пользователей из региона EU;
- Назначение теста: тестирование изменений, связанных с внедрением улучшенной рекомендательной системы;
- Ожидаемый эффект: за 14 дней с момента регистрации в системе пользователи покажут улучшение каждой метрики не менее, чем на 5 процентных пунктов: конверсии в просмотр карточек товаров — событие product_page, просмотры корзины — product_cart, покупки — purchase .

**ОТКРЫТИЕ ФАЙЛОВ И УЗУЧЕНИЕ ИНФОРМАЦИИ**

In [0]:
#Импорт библиотек
import pandas as pd
import datetime as dt
from datetime import datetime, timedelta

import numpy as np
from plotly import graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import warnings
from statsmodels.stats.proportion import proportions_ztest

sns.set()
warnings.filterwarnings('ignore')
matplotlib.style.use('seaborn-pastel')
%config InlineBackend.figure_format = 'retina'

In [1]:
pip install -U kaleido

In [2]:
df_marketing_events = pd.read_csv('https://code.s3.yandex.net/datasets/ab_project_marketing_events.csv')

In [3]:
df_marketing_events.head()

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11


In [4]:
df_marketing_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 580.0+ bytes


In [5]:
df_new_users = pd.read_csv('https://code.s3.yandex.net/datasets/final_ab_new_users.csv')

In [6]:
df_new_users.head()

,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone


In [7]:
df_new_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61733 entries, 0 to 61732
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     61733 non-null  object
 1   first_date  61733 non-null  object
 2   region      61733 non-null  object
 3   device      61733 non-null  object
dtypes: object(4)
memory usage: 1.9+ MB


In [ ]:
df_ab_events = pd.read_csv('https://code.s3.yandex.net/datasets/final_ab_events.csv')

In [ ]:
df_ab_events.head()

In [ ]:
df_ab_events.info()

In [ ]:
df_users = pd.read_csv('https://code.s3.yandex.net/datasets/final_ab_participants.csv')

In [ ]:
df_users.head()

In [ ]:
df_users.info()

**ПРЕДОБРАБОТКА ДАННЫХ**

**Первая таблица ab_project - календарь маркетинговых событий на 2020 год.**

In [ ]:
# преобразуем столбцы в нужный формат даты
df_marketing_events['start_dt'] = pd.to_datetime(df_marketing_events['start_dt'], format='%Y-%m-%d')
df_marketing_events['finish_dt'] = pd.to_datetime(df_marketing_events['finish_dt'], format='%Y-%m-%d')
df_marketing_events.info()

In [ ]:
# проверим дубликаты 
df_marketing_events.duplicated().sum()

In [ ]:
# проверим пропуски
df_marketing_events.isna().sum()

In [ ]:
# проверим
df_marketing_events.head()

**Вторая таблица final_ab_new - все пользователи, зарегистрировавшиеся в интернет-магазине в период с 7 по 21 декабря 2020 года.**

In [ ]:
# преобразуем столбцы first_date в нужный формат даты
df_new_users['first_date'] = pd.to_datetime(df_new_users['first_date'], format='%Y-%m-%d')
df_new_users.info()

In [ ]:
# проверили дубликаты
df_new_users.duplicated().sum()

In [ ]:
# проверили пропуски
df_new_users.isna().sum()

In [ ]:
# проверяем
df_new_users.head()

**Третья таблица final_ab_events - все события новых пользователей в период с 7 декабря 2020 по 4 января 2021 года.**

In [ ]:
# преобразуем столбцы event_dt в нужный формат даты
df_ab_events['event_dt'] = pd.to_datetime(df_ab_events['event_dt'], format='%Y-%m-%d')
df_ab_events.info()

In [ ]:
# Проверяем дубликаты
df_ab_events.duplicated().sum()

In [ ]:
# проверим пропуски
df_ab_events.isna().sum()

In [ ]:
df_ab_events.head()

In [ ]:
print ('наиболее ранее событие произошло', df_ab_events['event_dt'].min())
print ('наиболее позднее событие произошло',df_ab_events['event_dt'].max())

События начинаются 2020-12-07 00:00:33 и заканчиваются 2020-12-30 23:36:33. Согласно условиям в final_ab_events хранятся действия новых пользователей в период с 7 декабря 2020 по 4 января 2021 года.

**Четвертая таблица - таблица участников тестов.**

In [ ]:
df_users.head()

In [ ]:
# проверяем дубликаты
df_users.duplicated().sum()

In [ ]:
# проверяем пропуски
df_users.isna().sum()

**Вывод:**
- Столбцы приведены к нужному формату.
- Дубликатов нет.
- Некоторые названия приведены к нижнему регистру.
- Обнаружны пропуски в столбце details в кол-ве 377577 шт., но носят естественный характер (не было покупки, нет и стоимости).
- Есть пересечения в событиях, с 25.12.2020 по 03.01.2021 в EU параллельно проводилась акция "Christmas&New Year Promo", которая могла как-то повлиять на наш тест.

**1. Оценка корректности проведения теста**

In [ ]:
# длительность теста
print('Анализируемый период, пользователи: ', df_new_users['first_date'].min(), 'по', df_new_users['first_date'].max())

In [ ]:
# по событиям
print('Анализируемый период, события: ', df_ab_events['event_dt'].min(), 'по', df_ab_events['event_dt'].max())

Дата запуска теста верная 2020-12-07. Исходя из ТЗ, тест должен длиться до 2021-01-04, нет данных за 5 дней, видимо новогодние праздники повлияли.

In [ ]:
# оценка пользователей
df_users.pivot_table(index='ab_test', columns='group', values='user_id', aggfunc='count')

Параллельно проводился тест по интерфейсу interface_eu_test.

In [ ]:
# смотрим на пересечение пользователей этих тестов
df_pivot_users = df_users.groupby('user_id')['ab_test'].agg('nunique').reset_index()
df_pivot_users[df_pivot_users['ab_test'] > 1].shape[0]

Пересечение по пользователям, которые принимали участие в двух тестах составляет 1602.

In [ ]:
# проверим равномерность распределения пользователей по тестовым группам 
df_users_clean = df_users[df_users['ab_test'] == 'recommender_system_test']
df_users_clean.groupby('group')['user_id'].count()

In [ ]:
print('Общее кол-во пользователей:', df_users_clean.shape[0])

In [ ]:
# проверим тезис из ТЗ: 15% новых пользователей из региона EU
# объединим с данными по новым пользователям
df_user_test = df_users_clean.merge(df_new_users)

df_user_test_cnt = df_user_test[df_user_test['region']=='EU'].shape[0]

eu_new_users_cnt = df_new_users[
                                (df_new_users['first_date'] >= '2020-12-07') &
                                (df_new_users['first_date'] <= '2020-12-21') &
                                (df_new_users['region'] == 'EU')].shape[0]
df_user_test = df_user_test[df_user_test['region']=='EU']
print('Кол-во новых пользователей из региона "EU": {:.1%}'.format(df_user_test_cnt/eu_new_users_cnt))

**Маркетинговые события**

In [ ]:
df_marketing_events.sort_values(by='start_dt')

В наш временной период попадают 2 акции. Акция Christmas&New Year Promo может оказать большее влияние, т.к. даты её проведения попадают в исследуемый период с 12 декабря - по 4 января, плюс эта акция проводится в регионе EU, именно этот регион преобладает в тесте.

**Проверим нет ли пользователей, участвующих в двух группах одновременно**

In [ ]:
# проверим присутствуют ли пользователи, входящие в обе группы А и В теста recommender_system_test
df_users.query('ab_test == "recommender_system_test"')['user_id'].duplicated().sum()

**Лайфтайм событий**

In [ ]:
# делаем фильтр до 21 декабря
new_df_new_users = df_new_users[df_new_users['first_date'] <= "2020-12-21 00:00:00"]
new_df_new_users['first_date'].max()

In [ ]:
# фильтр по нужному нам тесту
recom_users = new_df_new_users[new_df_new_users['user_id'].isin(df_users.query('ab_test == "recommender_system_test"')['user_id'])]

In [ ]:
# объединяем таблицы 
recom_users_ab = recom_users.merge(df_users.query('ab_test == "recommender_system_test"'), on='user_id')
display(recom_users_ab.user_id.nunique())
recom_users_ab = recom_users_ab.merge(df_ab_events, how='left', on='user_id')
display(recom_users_ab.user_id.nunique())

In [ ]:
# делаем фильтр в 14 дней

#горизонт событий - 14 дней, 
#последняя дата наблюдения - 4 января, 
#последняя подходящая для анализа дата (4 января - 14 дней) 

horizon_days = 14
observation_date = datetime(2021, 1, 4).date()
last_suitable_acquisition_date = observation_date - timedelta(days = horizon_days - 1)

#фильтруем по дате последнего набора пользователей 
#добавляем лайфтайм пользователя
#делаем по нему фильтр в 14 дней 
display(recom_users_ab.user_id.nunique())
test_1 = recom_users_ab
ecom_users_ab = recom_users_ab.query('event_dt <= @last_suitable_acquisition_date')
display(recom_users_ab.user_id.nunique())
recom_users_ab['lifetime'] = (recom_users_ab['event_dt'] - recom_users_ab['first_date']).dt.days
recom_users_ab = recom_users_ab.query('lifetime < 14')
display(recom_users_ab.user_id.nunique())

In [ ]:
recom_users_ab.lifetime.nunique()

Поскольку нас интересуют только пользователи, которых составляет 14 дней с момента регистрации, из датасета исключим пользователей, которые зарегистрировались менее чем за 14 дней до окончания теста.

**Выводы:**

- Есть пользователи, которые появились после остановки набора 2020-12-21.
- Исходя из ТЗ, тест должен длиться до 2021-01-04, нет данных за 5 дней, видимо новогодние праздники повлияли.
- Параллельно с нашим проводился тест по интерфейсу interface_eu_test.
- Пересечение по пользователям, которые принимали участие в 2-х тестах составляет 1602.
- Есть пересечения в событиях, с 25.12.2020 по 03.01.2021 в EU параллельно проводилась акция "Christmas&New Year Promo", которая могла как-то повлиять на наш тест.
- Пересекается. Принимаем решение что пересечение не критично и условие ТЗ выполняется полностью.
- Аудитория сформирована правильно, кол-во новых пользователей из "EU" 15%.
- Также проверила количество пользователей, попавших в обе группы их 0.
- Исключила пользователей, которые зарегистрировались менее чем за 14 дней до окончания теста.

**2. Анализ и оценка метрик**

Воронка состоит из 4х этапов:

- login - пользователь вошёл на сайт
- product_page - посмотрел товар
- product_cart - попал в корзину
- purchase - совершил покупку

In [ ]:
df_total_users = recom_users_ab.groupby('event_name')['user_id'].nunique().sort_values(ascending=False).to_frame().reset_index()\
        .rename(columns={'user_id': 'total_users'})


total_users = recom_users_ab['user_id'].nunique()

df_total_users['percent'] = round ((df_total_users['total_users'] / total_users*100),2)
df_total_users['percent'] = df_total_users['percent'].astype('str')+'%'
df_total_users

Только 63 % от всех залогинившихся пользователей посмотрели страницу продукта и около 30% совершили покупку. При этом судя по данным для покупки вовсе не обязательно добавлять товар в корзину.

Визуализируем наши расчеты.

In [ ]:
fig = go.Figure()

fig.add_trace(go.Funnel(
    name = 'All Users',
    y = df_total_users['event_name'],
    x = df_total_users['total_users'],
       marker = {"color": ["deepskyblue", "lightsalmon", "tan", "teal"],
               "line": {"width": [4, 2, 3, 1],
               "color": ["wheat", "wheat", "blue", "wheat"]}},
   connector = {"line": {"color": "royalblue", "dash": "dot", "width": 2}}
    ))
fig.update_layout(title_text = 'Воронка пользователей')
fig.show()

По количеству всех событий пользователей тоже видно, что из 100% лишь 30% совершают покупку, но корзину посещает лишь 29%.

In [ ]:
funnel_A = recom_users_ab.query("group == 'A'").groupby('event_name')['user_id'].nunique().sort_values(ascending=False).to_frame().reset_index()\
        .rename(columns={'user_id': 'count'})
funnel_B = recom_users_ab.query("group == 'B'").groupby('event_name')['user_id'].nunique().sort_values(ascending=False).to_frame().reset_index()\
        .rename(columns={'user_id': 'count'})

In [ ]:
funnel_B

In [ ]:
funnel_A

In [ ]:

new_index = {2: 3, 3: 2}
funnel_A = funnel_A.rename(new_index).sort_index()
funnel_B = funnel_B.rename(new_index).sort_index()

fig = go.Figure()

fig.add_trace(go.Funnel(
    name = 'Group A',
    y = funnel_A['event_name'],
    x = funnel_A['count'],
    ))

fig.add_trace(go.Funnel(
    name = 'Group B',
    y = funnel_B['event_name'],
    x = funnel_B['count'],
    ))

fig.update_layout(title_text = 'Воронка пользователей')
fig.show()
fig.show()

В первую очередь видно, неравномерное распределение пользователей между группами.
- немногим менее половины клиентов после авторизации не доходят до карточки товара;
- от просмотра страницы покупки на следующий шаг попадают также только около половины пользователей с предыдущего шага.

Видимо покупку можно совершить не через корзину. Потому что корзину посетили 29%, а заказ оформили 31%. То есть из 100% изначальных лишь 31% завершает покупку.

По всей видимости есть пользователи пришедшие "из вне". Точнее пользователей с покупками больше чем тех, кто просмотрел корзину.

In [ ]:
# очищенные данные по действиям пользователей
df_ab_events_clean = df_ab_events[df_ab_events['user_id'].isin(df_users_clean['user_id'])]
# создадим фрейм с событиями
df_events =  df_ab_events_clean.merge(df_users_clean)
# выделим дату
df_events['report_dt'] = df_events['event_dt'].dt.date

t = df_events.groupby(['user_id', 'report_dt', 'group']).agg(events = ('event_name', 'count'))
tt = t.pivot_table(index='report_dt',
              columns='group',
              values='events',
              aggfunc='mean'
             ).reset_index()

dates_ = pd.date_range(df_events['event_dt'].min(), df_events['event_dt'].max(), freq='D')

plt.figure(figsize=(11, 6))
plt.ylim((1, 3))
plt.title('Динамика среднего кол-ва событий по группам теста')
sns.lineplot(x='report_dt', y='A', data=tt, label='A')
sns.lineplot(x='report_dt', y='B', data=tt, label='B')
plt.xlabel('')
plt.ylabel('')
plt.xticks(dates_, rotation=45)
plt.legend()
plt.show()

По новой платёжной воронки динамика ниже примерно на четверть.

In [ ]:
df_dinamic = df_events.pivot_table(index='report_dt', columns='group', values='user_id', aggfunc='count')
plt.figure(figsize = (11,6))
fig = sns.lineplot(data=df_dinamic)
plt.title('Динамика событий по группам',fontsize=13)
plt.xlabel('день',fontsize=13)
plt.ylabel('события, шт',fontsize=13)
plt.show()

Дата остановки набора новых пользователей - 2020-12-21, в этот день совершено больше всего дейсвий, затем график идет на спад. Число событий сильно отличается, если учесть что среднее не так сильно отличается, сильно увеличивается число пользователей.

In [ ]:
event_by_user = recom_users_ab.groupby(['group', 'user_id'], as_index=False).agg({'event_name':'count'})

for i in ['A', 'B']:
    print('Среднее количество событий на пользователя в группе {} составляет {}\n'.format(
        i,
        event_by_user.query('group == @i')['event_name'].median()
    ))

In [ ]:
event_by_user 

In [ ]:
events_by_day = event_by_user.groupby('group')['event_name'].count().reset_index()
events_by_day.head()

In [ ]:
fig = go.Figure([go.Bar(x=events_by_day['group'], y=events_by_day['event_name'], text = events_by_day['event_name'],
                      textposition='auto')])
fig.update_layout(title_text='Гистограмма событий по дням',
                 xaxis_title_text='Дата',
                 yaxis_title_text='Количество',
                 width=700, height=300)

fig.show()

В среднем в группе А чаще, чем в В встречаются пользователи с количеством событий.

**3. Анализ A/B теста**

In [ ]:
# поиск аномалий
#df_events.groupby('user_id')['group'].count()
plt.figure(figsize = (11,6))
ax = sns.boxplot(x=df_events.groupby('user_id')['group'].count())
plt.title('Распределение событий пользователям')
plt.xlabel('Кол-во событий')
plt.show()
# 95 и 99 перцентиль
print( np.percentile(df_events.groupby('user_id')['group'].count(), [95, 99]) )

Выбросом будем считать более 15.

In [ ]:
# почистим данные
#df_bad_user = df_events.groupby('user_id')['group'].count().reset_index()
#df_bad_user = df_bad_user[df_bad_user['group'] >= 15]
#blackus_list = df_bad_user['user_id'].tolist()
#df_events = df_events[~df_events['user_id'].isin(blackus_list)]
#print('Столько пользователей исключили из расчёта: ', len(blackus_list) )

**Статистическая разница долей z-критерий.**

In [ ]:
def calc_z_test(stage_0,stage_1):
    '''
    Функция проверяет z-критерием статистическую разницу долей в группах А и В при конверсии от одного этапа к другому.
    На вход принимает значения событий из event_name, в последовательности, которой хочешь посчитать тест
    '''
    alpha = .05 / 3
    event_nm0 = stage_0
    event_nm1 = stage_1
    df_res = df_events.query('event_name == @event_nm0 | event_name == @event_nm1').pivot_table(index = 'group', 
                                                                                              columns = 'event_name', 
                                                                                               values = 'user_id',
                                                                                               aggfunc = 'nunique')
    df_res['rate'] = df_res[event_nm1] / df_res[event_nm0]
    cnt0 = df_res[event_nm1].tolist()
    cnt1 = df_res[event_nm0].tolist()
    
    pvalue = proportions_ztest(cnt0, cnt1, value = 0)[1]
    print(cnt0, cnt1)
    print(f'Изменение воронки {event_nm0} -> {event_nm1}')
    print('p-value: {}'.format(pvalue))
    if pvalue > alpha:
        print('Не удалось отвергнуть нулевую гипотезу: между групп A и B отсутствует разница в конверсии')
    else: 
        print('Нулевая гипотеза отвергается: между группами A и B есть стат.различия в конверсии')
    
    df_res.reset_index(inplace=True)
    before = float(df_res[df_res['group'] == 'A']['rate'])
    after = float(df_res[df_res['group'] == 'B']['rate'])
    print('Изменение конверсии : {:.2%}'.format((after - before)/before ))

Согласно ТЗ рассмотрим по порядку:

1. Конверсии в просмотр карточек товаров — событие product_page

H0: Между группами A и B нет различий в конверсии в покупку

H1: Между группами A и B есть различие конверсии в покупку

In [ ]:
calc_z_test('login', 'product_page')

2. Конверсии просмотры корзины — product_cart

H0: Между группами A и B нет различий в конверсии в просмотр корзины

H1: Между группами A и B есть различие конверсии в просмотр корзины

In [ ]:
calc_z_test('login', 'product_cart')

3. Конверсии покупки — purchase

H0: Между группами A и B нет различий в конверсии в покупку

H1: Между группами A и B есть различие конверсии в покупку

In [ ]:
calc_z_test('login', 'purchase')

**Общий вывод:**

Выводы:

Есть пользователи, которые появились после остановки набора 2020-12-21.
Исходя из ТЗ, тест должен длиться до 2021-01-04, нет данных за 5 дней, видимо новогодние праздники повлияли.
Параллельно с нашим проводился тест по интерфейсу interface_eu_test.
Пересечение по пользователям, которые принимали участие в 2-х тестах составляет 1602.
Есть пересечения в событиях, с 25.12.2020 по 03.01.2021 в EU параллельно проводилась акция "Christmas&New Year Promo", которая могла как-то повлиять на наш тест.
Пересекается. Принимаем решение что пересечение не критично и условие ТЗ выполняется полностью.
Аудитория сформирована правильно, кол-во новых пользователей из "EU" 15%.
Также проверила количество пользователей, попавших в обе группы их 0.
Исключила пользователей, которые зарегистрировались менее чем за 14 дней до окончания теста.

Были выявлены след.нарушения:

- Проводилось 2 теста одновременно;
- Разница в кол-ве участников;
- Исходя из ТЗ, тест должен длиться до 2021-01-04, нет данных за 5 дней, видимо новогодние праздники повлияли.

Проверены гипотезы на равенство долей:

- конверсии -> между группами A и B есть стат.различия в конверсии;
- конверсии -> между групп A и B отсутствует разница в конверсии;
- конверсия -> между групп A и B отсутствует разница в конверсии.

В результате теста выявлено что новая рекомендательная система не позволила улучшить целевые метрики.

При проведении тестирования наблюдаем, что в группу B попало меньше пользователей, чем в группу А. При идеальном делении 50\50. Также наше тестирование попало под допольно длительную маркетинговую кампанию. Что уже считает некорректным решением при проведении тестирования.

Рекомендации:
Нужно подумать о "клиентском пути" на сайте, дополнительно проанализировать действия и время нахождения клиента на сайте. Довольно странно, что авторизовавшись половина клиентов даже не посмотрела товар. Возможно есть технические проблемы в целом или проблемы на каких - то устройствах.